<a href="https://colab.research.google.com/github/PhantomXV7/PhantomXV7/blob/main/Amazon_ML_hackathon_TestCleaner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#https://drive.google.com/file/d/1Lqia2upHkEE9Bu67eMJqTskelN8JV4__/view?usp=share_link
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1Lqia2upHkEE9Bu67eMJqTskelN8JV4__' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1Lqia2upHkEE9Bu67eMJqTskelN8JV4__" -O test.ftr && rm -rf /tmp/cookies.txt
!pip install torch
!pip install pytorch_lightning 
!pip install transformers
!pip install contractions
!pip install torchvision

--2023-04-21 18:06:53--  https://docs.google.com/uc?export=download&confirm=t&id=1Lqia2upHkEE9Bu67eMJqTskelN8JV4__
Resolving docs.google.com (docs.google.com)... 209.85.146.138, 209.85.146.101, 209.85.146.100, ...
Connecting to docs.google.com (docs.google.com)|209.85.146.138|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0k-28-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/su1g4rkkrkapljbsojan79br8h64p7hl/1682100375000/12978060410603387978/*/1Lqia2upHkEE9Bu67eMJqTskelN8JV4__?e=download&uuid=5b7186b4-ffef-4088-b0fd-8c1bd9aa93b4 [following]
--2023-04-21 18:06:53--  https://doc-0k-28-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/su1g4rkkrkapljbsojan79br8h64p7hl/1682100375000/12978060410603387978/*/1Lqia2upHkEE9Bu67eMJqTskelN8JV4__?e=download&uuid=5b7186b4-ffef-4088-b0fd-8c1bd9aa93b4
Resolving doc-0k-28-docs.googleusercontent.com (doc-0k-28-docs.googleusercontent.com)... 108.177.121.132,

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gc
from tqdm import tqdm

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%%time
test = pd.read_feather("test.ftr") 

CPU times: user 1.58 s, sys: 2.71 s, total: 4.28 s
Wall time: 3.89 s


In [5]:
len(test)

734736

In [6]:
test.isna().sum()

PRODUCT_ID         0
PRODUCT_TYPE_ID    0
td                 0
bd                 0
dtype: int64

In [7]:
test.head()

,PRODUCT_ID,PRODUCT_TYPE_ID,td,bd
0,604373,6142,Manuel d'Héliogravure Et de Photogravure En Re...,
1,1729783,1622,DCGARING Microfiber Throw Blanket Warm Fuzzy P...,[QUALITY GUARANTEED: Luxury cozy plush polyest...
2,1871949,7540,I-Match Auto Parts Front License Plate Bracket...,"[Front License Plate Bracket Made Of Plastic,D..."
3,1107571,12442,PinMart Gold Plated Excellence in Service 1 Ye...,[Available as a single item or bulk packed. Se...
4,624253,6318,"Visual Mathematics, Illustrated by the TI-92 a...",


In [8]:
import re
from nltk.corpus import stopwords
from wordcloud import WordCloud, STOPWORDS
import spacy
import contractions
import nltk
nltk.download('stopwords')

def textCleaner(mystring):
    s = ''
    for x in mystring.split():
        s = s+' '+ contractions.fix(x) 
    #re.sub(r'(!|.)1+', '', text1) 
    return re.sub(r"^\W+", "", s)

# spacy.prefer_gpu()
# nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
# doc = nlp(newstring)
# " ".join([token.lemma_ for token in doc])
def textCleaner_adv(mystring):
    newstring = " ".join(x.lower() for x in mystring.split())
    
    newstring = newstring.replace('[^\w\s]','')
    
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags 
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    newstring = emoji_pattern.sub(r'', newstring)
    
    stop = stopwords.words('english')
    newstring = " ".join(x for x in newstring.split() if x not in stop)
    
    # doc = nlp(newstring)
    # " ".join([token.lemma_ for token in doc])
    return newstring

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [9]:
from tqdm.notebook import tqdm
tqdm.pandas()

In [10]:
%%time
for col in ['td', 'bd']:
    test[col] = test[col].progress_apply(lambda x: textCleaner_adv(textCleaner(x)))

  0%|          | 0/734736 [00:00<?, ?it/s]

  0%|          | 0/734736 [00:00<?, ?it/s]

CPU times: user 17min 30s, sys: 36.8 s, total: 18min 7s
Wall time: 18min 39s


In [11]:
test.head()

,PRODUCT_ID,PRODUCT_TYPE_ID,td,bd
0,604373,6142,manuel d'héliogravure et de photogravure en re...,
1,1729783,1622,dcgaring microfiber throw blanket warm fuzzy p...,quality guaranteed: luxury cozy plush polyeste...
2,1871949,7540,i-match auto parts front license plate bracket...,"front license plate bracket made plastic,direc..."
3,1107571,12442,pinmart gold plated excellence service 1 year ...,available single item bulk packed. select quan...
4,624253,6318,"visual mathematics, illustrated ti-92 ti-89",


In [12]:
test.to_feather('test_cleaned.ftr')

In [13]:
import locale
import shutil

# Set locale to UTF-8
locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')

# Copy file to destination
shutil.copy2('/content/test_cleaned.ftr', '/content/drive/MyDrive/')


'/content/drive/MyDrive/test_cleaned.ftr'

In [14]:
!cp -r '/content/test_cleaned.ftr' /content/drive/MyDrive/

In [15]:
test.isna().sum()



PRODUCT_ID         0
PRODUCT_TYPE_ID    0
td                 0
bd                 0
dtype: int64